## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/ssteffen/desktop/Sam/Vanderbilt Boot Camp/MyRepo/Module 6_APIs/Challenge/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,63.99,80,49,12.21,scattered clouds
1,1,Chuy,UY,-33.6971,-53.4616,45.63,91,1,7.23,clear sky
2,2,Angoche,MZ,-16.2325,39.9086,70.05,74,0,7.83,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,75,100,17.27,overcast clouds
4,4,Hope,US,33.6671,-93.5916,84.25,65,7,8.84,clear sky


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Hope,US,33.6671,-93.5916,84.25,65,7,8.84,clear sky
8,8,Fort-Shevchenko,KZ,44.5086,50.2630,85.86,36,5,11.70,clear sky
11,11,Lingao,CN,19.9103,109.6859,87.42,71,25,9.51,scattered clouds
12,12,Bengkulu,ID,-3.8004,102.2655,83.30,70,32,2.39,scattered clouds
17,17,Sabang,ID,5.8933,95.3214,82.54,78,99,14.65,light rain
24,24,Mecca,SA,21.4267,39.8261,83.17,52,73,2.57,broken clouds
25,25,Marsh Harbour,BS,26.5412,-77.0636,83.14,76,43,11.92,scattered clouds
27,27,Nhulunbuy,AU,-12.2333,136.7667,77.61,61,40,21.85,scattered clouds
30,30,Wewak,PG,-3.5534,143.6268,84.54,71,100,4.52,overcast clouds
32,32,The Valley,AI,18.2170,-63.0578,82.26,74,100,20.76,overcast clouds


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                240
City                   240
Country                240
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hope,US,84.25,clear sky,33.6671,-93.5916,
8,Fort-Shevchenko,KZ,85.86,clear sky,44.5086,50.2630,
11,Lingao,CN,87.42,scattered clouds,19.9103,109.6859,
12,Bengkulu,ID,83.30,scattered clouds,-3.8004,102.2655,
17,Sabang,ID,82.54,light rain,5.8933,95.3214,
24,Mecca,SA,83.17,broken clouds,21.4267,39.8261,
25,Marsh Harbour,BS,83.14,scattered clouds,26.5412,-77.0636,
27,Nhulunbuy,AU,77.61,scattered clouds,-12.2333,136.7667,
30,Wewak,PG,84.54,overcast clouds,-3.5534,143.6268,
32,The Valley,AI,82.26,overcast clouds,18.2170,-63.0578,


In [14]:
# This cell is used to test the JSON data to determine the file path for step 6f
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "key": g_key,
#     "location": "33.6671, -93.5916"}
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make request and get the JSON data from the search.
# hotels = requests.get(base_url, params=params).json()
# hotels
# hotel_name = hotels["results"][0]["name"]

In [15]:
#count the number of results from the above json script
# len(hotels["results"])

In [16]:
# 6a. Set parameters to search for hotels within 5000 meters.
hotel_params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    hotel_params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=hotel_params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hope,US,84.25,clear sky,33.6671,-93.5916,Best Western Hope
8,Fort-Shevchenko,KZ,85.86,clear sky,44.5086,50.2630,Oasis hotel Bautino
11,Lingao,CN,87.42,scattered clouds,19.9103,109.6859,Lingao Ditai Inn
12,Bengkulu,ID,83.30,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
17,Sabang,ID,82.54,light rain,5.8933,95.3214,Hotel Citra
24,Mecca,SA,83.17,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
25,Marsh Harbour,BS,83.14,scattered clouds,26.5412,-77.0636,Abaco Beach Resort
27,Nhulunbuy,AU,77.61,scattered clouds,-12.2333,136.7667,East Arnhem Real Estate
30,Wewak,PG,84.54,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
32,The Valley,AI,82.26,overcast clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort


In [17]:
# 7. Drop the rows where there is no Hotel Name.
#import NumPy dependency to use np.nan
import numpy as np

#Replace the empty cells with Nan using .loc
hotel_df.loc[(hotel_df["Hotel Name"] == ""), ["Hotel Name"]] = np.nan

#Use dropna() to eliminate the columns containing NaN
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hope,US,84.25,clear sky,33.6671,-93.5916,Best Western Hope
8,Fort-Shevchenko,KZ,85.86,clear sky,44.5086,50.2630,Oasis hotel Bautino
11,Lingao,CN,87.42,scattered clouds,19.9103,109.6859,Lingao Ditai Inn
12,Bengkulu,ID,83.30,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
17,Sabang,ID,82.54,light rain,5.8933,95.3214,Hotel Citra
24,Mecca,SA,83.17,broken clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
25,Marsh Harbour,BS,83.14,scattered clouds,26.5412,-77.0636,Abaco Beach Resort
27,Nhulunbuy,AU,77.61,scattered clouds,-12.2333,136.7667,East Arnhem Real Estate
30,Wewak,PG,84.54,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
32,The Valley,AI,82.26,overcast clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description and Max Temp</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))